# Cluster bulk list of keywords with valueserp

In [220]:
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile
import requests

In [226]:
# inputs

api_key = "add key here"

common_urls = 2

In [ ]:
# create batch upload file



# include the fist 30 not just first page

In [1]:
# upload batch file to valueserp and get batch_id

batch_id = '903576B8'

In [4]:
# start the batch job

import requests

params = {
  'api_key': api_key
}

api_result = requests.get('https://api.valueserp.com/batches/'+batch_id+'/start', params)

api_response = api_result.json()

print("Batch started.")


Batch started.


In [6]:
# check for job completed


import requests

params = {
  'api_key': api_key
}

api_result = requests.get('https://api.valueserp.com/batches/'+batch_id, params)

api_response = api_result.json()

print("Batch Name: ", api_response['batch']['name'])

api_response

Batch Name:  test


{'request_info': {'success': True},
 'batch': {'id': '903576B8',
  'created_at': '2023-02-11T03:57:53.219Z',
  'last_run': '2023-02-11T04:06:05.773Z',
  'name': 'test',
  'schedule_type': 'manual',
  'enabled': True,
  'status': 'idle',
  'searches_total_count': 47,
  'searches_page_count': 1,
  'credits_required': 47,
  'next_result_set_id': 2,
  'results_count': 1,
  'priority': 'normal',
  'destination_ids': [],
  'notification_email': 'alton@frontanalytics.com',
  'notification_as_json': True,
  'notification_as_jsonlines': False,
  'notification_as_csv': True,
  'notification_csv_fields': 'search.q,organic_results.position,organic_results.title,organic_results.link',
  'searches_type': 'web',
  'searches_per_type_count': {'web': 47},
  'searches_type_locked': True}}

In [320]:
# get the link for the download

import requests

params = {
  'api_key': api_key
}

api_result = requests.get('https://api.valueserp.com/batches/'+batch_id+'/results/1/csv', params)

api_response = api_result.json()

In [321]:


content = requests.get(url)
zf = ZipFile(BytesIO(content.content))

for item in zf.namelist():
    print("File in zip: "+  item)

# find the first matching csv file in the zip:
match = [s for s in zf.namelist() if ".csv" in s][0]
# the first line of the file contains a string - that line shall de     ignored, hence skiprows


File in zip: Batch_Results_903576B8_1_Page_1_002c0139d00c9dd8e4d1b13c504517942bd20489.csv


In [366]:
# save the cluster file
df = pd.read_csv(zf.open(match), low_memory=False)

delim = ";;"

# make dataframe from the serp lists
df = df[['result.organic_results.link', 'search.q', 'result.organic_results.position']]
df = df.rename(columns={'result.organic_results.link':'link', 'search.q':'query', 'result.organic_results.position':'position'})

# Clean input queries
df['query'] = df['query'].str.strip()

# distinct queries
queries = set(list(df['query']))

# make position dataframe
df_position = df[["position", "query", "link"]].copy()

df = df.sort_values(by="query", ascending=True)
df = df.groupby('link')['query'].apply(delim.join).reset_index()
df["temp_count"] = (df["query"].str.count( delim ) + 1)
df = df[df["temp_count"] >= common_urls]

# keep only the longest version of each keyword group
list1 = df['query']
substrings = {w1 for w1 in list1 for w2 in list1 if w1 in w2 and w1 != w2}
longest_str = set(list1) - substrings
longest_str = list(longest_str)
df = df[df['query'].isin(longest_str)]

# drop any cluster groups under the minimum threshold
df['temp_count'] = df['query'].map(df.groupby('query')['query'].count())
df = df[df["temp_count"] >= common_urls]

# make the cluster column & delete helper column
df['serp_cluster'] = df['query']

# explode the keyword list
df['query'] = df['query'].str.split( delim )
df = df.explode('query')

# rename the cluster name to the shortest keyword
df['temp_count'] = df['query'].astype(str).map(len)
df = df.sort_values(by="temp_count", ascending=True)
df['serp_cluster'] = df.groupby('serp_cluster')['query'].transform('first')

del df['temp_count']
del df['link']

df.drop_duplicates(subset="query", inplace=True)

# add in no cluster results
df_no_cluster = pd.DataFrame(None)
queries_de_duped = list(set(queries))
df_no_cluster['query'] = queries_de_duped
df_no_cluster['serp_cluster'] = "zzz_no_cluster"
df_cluster = pd.concat([df_no_cluster, df])
df_cluster = df_cluster[['serp_cluster', 'query']]
df_cluster = df_cluster.sort_values(by="serp_cluster", ascending=True)
df_cluster.drop_duplicates(subset=['query'], keep="first", inplace=True)


df_cluster.groupby(['serp_cluster','query']).first()#.reset_index().groupby(['serp_cluster','query','temp_count']).count()

Empty DataFrame
Columns: []
Index: [(bike lanes chicago, bike lanes chicago), (bike lanes chicago, bike lanes in chicago), (bike lanes chicago, chicago bike lanes), (crybaby hill tulsa, cry baby hill tulsa), (crybaby hill tulsa, crybaby hill tulsa), (des moines bike lanes, des moines bike lanes), (lanes mashpee, lanes mashpee), (lanes mashpee, the lanes mashpee), (los angeles bike lanes, bike lanes in los angeles), (los angeles bike lanes, bike lanes los angeles), (los angeles bike lanes, los angeles bike lanes), (miami bike lanes, bike lanes miami), (miami bike lanes, miami beach bike lanes), (miami bike lanes, miami bike lanes), (philadelphia bike lanes, bike lanes in philadelphia), (philadelphia bike lanes, philadelphia bike lanes), (salt lake city bike lanes, bike paths salt lake city), (salt lake city bike lanes, salt lake city bike lanes), (zzz_no_cluster, 9th and 9th bike ride), (zzz_no_cluster, alhambra bike rides), (zzz_no_cluster, altona bike rides), (zzz_no_cluster, aspinwall bike rides), (zzz_no_cluster, atlanta bike lanes), (zzz_no_cluster, bella villa bike rides), (zzz_no_cluster, bellemeade bike rides), (zzz_no_cluster, bellflower bike rides), (zzz_no_cluster, bike lanes dallas), (zzz_no_cluster, bike lanes dublin), (zzz_no_cluster, bike lanes san diego), (zzz_no_cluster, bike paths appleton wi), (zzz_no_cluster, critical mass nyc), (zzz_no_cluster, jersey city bike lanes), (zzz_no_cluster, lampeter bike rides), (zzz_no_cluster, ledgeview lanes), (zzz_no_cluster, malta bike routes), (zzz_no_cluster, meadowbrook farm bike rides), (zzz_no_cluster, millbourne bike rides), (zzz_no_cluster, millbrae bike rides), (zzz_no_cluster, millwood bike rides), (zzz_no_cluster, nashville bike lanes), (zzz_no_cluster, north braddock bike rides), (zzz_no_cluster, owyhee lanes), (zzz_no_cluster, phoenix bike lanes), (zzz_no_cluster, wagner lanes), (zzz_no_cluster, wesley hills bike rides)]

In [369]:
# save the clusters and queries
df_cluster.groupby(['serp_cluster','query']).first().reset_index().to_csv("clustered_queries.csv")